<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week06/pyspark/Pyspark_Structure_Streaming_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#!wget https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz
!pip install findspark
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [4]:
%%capture
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.3.1/spark-sql-kafka-0-10_2.12-3.3.1.jar"
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.1.3/spark-streaming-kafka-0-10_2.12-3.1.3.jar"

In [46]:
%mkdir /content/checkpoint2

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.3,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell'

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Read stream from kafka

In [36]:
spark = SparkSession.builder.appName("readFromKafka").master("local").getOrCreate()

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092") \
  .option("subscribe", "my-first-topic") \
  .option("startingOffsets", "earliest") \
  .load()

df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
df.printSchema()


root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



# Write stream to kafka

In [38]:
#psDF = df.selectExpr("CAST(value AS STRING)")
#df.selectExpr("firstname")

schema1 = StructType([StructField('id', IntegerType(), True), \
      StructField('firstname', StringType(), True), \
      StructField('middlename', StringType(), True), \
      StructField('lastname', StringType(), True), \
      StructField('dob_year', IntegerType(), True), \
      StructField('dob_month', IntegerType(), True),
      StructField('profession', StringType(), True),
      StructField('gender', StringType(), True),
      StructField('salary', IntegerType(), True)])

person = df \
    .select(from_json(df.value, schema1).alias("data")) \
    .select("data.*")

In [39]:
person.printSchema()

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob_year: integer (nullable = true)
 |-- dob_month: integer (nullable = true)
 |-- profession: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [48]:
df = person.selectExpr("firstname")

In [ ]:
ds = df \
  .selectExpr("CAST(firstname AS STRING) AS key", "to_json(struct(*)) AS value") \
  .writeStream \
  .format("kafka") \
  .trigger(continuous='2 second') \
  .option("kafka.bootstrap.servers", "ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092") \
  .option("topic", "my-first-topic5") \
  .option("checkpointLocation", "/content/checkpoint") \
  .start() \
  .awaitTermination()